In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('D:/学习/集思项目/final project/train.csv')
X = pd.read_csv(r'D:/学习/集思项目/final project/tfidf2.csv',header = None)

In [3]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,34634,34635,34636,34637,34638,34639,34640,34641,34642,34643
0,4.482953,30.570149,18.022587,6.971426,5.800844,8.37292,4.12232,2.206366,12.935958,2.680922,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,2.206366,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X = np.array(X)
Y = data[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']]
Y = Y.head(10000)
Y = np.array(Y)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [6]:
X_sum = np.row_stack((X_train, X_test))
Y_sum = np.row_stack((y_train, y_test))

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

X_sparse = sparse.csr_matrix(X_sum)

In [8]:
from sklearn.metrics import euclidean_distances

distances = euclidean_distances(X_sparse)

In [9]:
def fit_ML_KNN(k, X, Y, distances):
    
    s = 1
    m = len(X)
    P1 = np.zeros((6,))
    Pe1 = np.zeros((6, k + 1))
    P0 = np.zeros((6,))
    Pe0 = np.zeros((6, k + 1))
    ind = np.argsort(distances)
    
    for i in range(6):
        y = 0
        for j in range(m):
            if Y[j][i] == 1:
                y = y + 1
        P1[i] = (s + y) / (s * 2 + m)
        P0[i] = 1 - P1[i]
    
    for i in range(6):
        c1 = np.zeros(k + 1)
        c0 = np.zeros(k + 1)
        for j in range(m):
            temp = 0
            for kk in range(1,k + 1):
                if Y[ind[j][kk]][i] == 1:
                    temp = temp + 1
            if Y[j][i] == 1:
                c1[temp] = c1[temp] + 1
            else:
                c0[temp] = c0[temp] + 1
        
        for l in range(k + 1):
            Pe1[i][l] = (s + c1[l]) / (s * (k + 1) + c1.sum())
            Pe0[i][l] = (s + c0[l]) / (s * (k + 1) + c0.sum())
            
    return P1, P0, Pe1, Pe0

In [28]:
def predict_ML_KNN(k, P1, P0, Pe1, Pe0, test_Data, Y, distances):
    
    beta = 1.5
    rtl = np.zeros((test_Data.shape[0], 6))
    m = test_Data.shape[0]
    predict_labels = np.zeros((m, 6))
    ind = np.argsort(distances)
    
    W1 = 1
    W2 = 0
    for i in range(k):
        W2 = W2 + 1 / pow(beta, i - 1)
    W1 = 1 / W2
    W2 = (W2 - 1) / W2
    
    for i in range(m):
        for j in range(6):
            temp = 0
            temp2 = 0
            y1 = 0
            y0 = 0
            f = 1
            count = 0
            while count < k:
                if ind[i + 8000][f] >= 8000:
                    f = f + 1
                    continue
                else:
                    count = count + 1
                    if count == 1:
                        temp2 = ind[i + 8000][f]
                    if Y[ind[i + 8000][f]][j] == 1:
                        temp = temp + 1
                    f = f + 1
            
            y1 = P1[j] * Pe1[j][temp]
            y0 = P0[j] * Pe0[j][temp]
            rtl[i][j] = P1[j] * Pe1[j][temp] / (P1[j] * Pe1[j][temp] + P0[j] * Pe0[j][temp])
            '''
            y1 = P1[j] * Pe1[j][temp] * W2 + Y[temp2][j] * W1
            y0 = P0[j] * Pe0[j][temp] * W2 + (1 - Y[temp2][j]) * W1
            rtl[i][j] = y1 / (y1 + y0)
            '''
            if y1 > y0:
                predict_labels[i][j] = 1
            else:
                predict_labels[i][j] = 0
    
    return predict_labels, rtl

In [29]:
def hloss(predict_label, true_label):
  #预测集行数等于验证集大小，预测集列数等于验证集label大小
    data_num = predict_label.shape[0]
    label_num = predict_label.shape[1]
    hloss = 0
    for i in range(data_num):
        bias = 0
        for j in range(label_num):
            if predict_label[i,j] != true_label[i,j]:
                bias += 1
        hloss += bias/label_num
    hloss = hloss/data_num
    return hloss



def one_error(predict_rt, true_label): 
    
    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    one_error_num = 0
    for i in range(data_num):
        rtl = list(predict_rt[i])
        max_index = rtl.index(max(rtl))
        if true_label[i,max_index] != 1:
            one_error_num += 1
    
    return one_error_num/data_num

def sort_both(list_rtl, list_target):
    for j in range(len(list_rtl)-1):
        for k in range(j,len(list_rtl)):
            if list_rtl[j] <= list_rtl[k]:
                temp1 =list_rtl[j]
                temp2 =list_target[j]
                list_rtl[j] = list_rtl[k]
                list_target[j] = list_target[k]
                list_rtl[k] = temp1 
                list_target[k] = temp2
    
    return list_rtl,list_target

def coverage(predict_rt, true_label):
    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    coverage1 = 0
    for i in range(data_num):
        max_rank = 0
        rtl = list(predict_rt[i])
        target = list(true_label[i])
        rtl, target = sort_both(rtl,target)

        for j in range(len(rtl)):
            if target[j] == 1:
                max_rank = j+1
        coverage1 += max_rank
    
    return (coverage1/data_num-1)


def rloss(predict_rt, true_label):
    #import pdb; pdb.set_trace()

    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    r_loss = 0
    for i in range(data_num):
        index_1 = []
        index_0 = []
        rank_loss = 0
        rtl = list(predict_rt[i])
        target = list(true_label[i])
        rtl, target = sort_both(rtl,target)
        for j in range(label_num):
            if target[j] == 1:
                index_1.append(j)
            else:
                index_0.append(j)
      
        m = len(index_1)
        n = len(index_0)
        for j in range(m):
            for k in range(n):
                if index_0[k] <= index_1[j]:
                    rank_loss += 1
      
        r_loss += rank_loss/(m*n)
    return r_loss/data_num
  

def avgprec(predict_rt, true_label):

    data_num = predict_rt.shape[0]
    label_num = predict_rt.shape[1]
    avg_prec = 0
    for i in range(data_num):
        index_1 = []
        temp_prec = 0
        rtl = list(predict_rt[i])
        target = list(true_label[i])
        rtl, target = sort_both(rtl,target)
        for j in range(label_num):
            if target[j] == 1:
                index_1.append(j)
        for k in range(len(index_1)):
            temp_prec += (k+1)/(index_1[k]+1)
      
        avg_prec += temp_prec/len(index_1)
    
    return avg_prec/data_num

In [30]:
import time
distances = euclidean_distances(X_sparse)
X_train_sparse = sparse.csr_matrix(X_train)
X_train_euclidean_distances = euclidean_distances(X_train_sparse)
true_labels = Y[8000:][:]
kk = 20
Hamming_Loss = np.zeros(kk)
One_Error = np.zeros(kk)
Coverage = np.zeros(kk)
Ranking_Loss = np.zeros(kk)
Average_Precision = np.zeros(kk)
count = -1
for k in np.arange(2,21,1):
    count = count + 1
    start_time = time.time()
    P1, P0, Pe1, Pe0 = fit_ML_KNN(k, X_train, y_train, X_train_euclidean_distances)
    predict_labels, predict_rtl = predict_ML_KNN(k, P1, P0, Pe1, Pe0, X_test, Y_sum, distances)
    end_time = time.time()
    Hamming_Loss[count] = hloss(predict_labels, true_labels)
    One_Error[count] = one_error(predict_rtl, true_labels)
    Coverage[count] = coverage(predict_rtl, true_labels)
    Ranking_Loss[count] = rloss(predict_rtl, true_labels)
    Average_Precision[count] = avgprec(predict_rtl, true_labels)
    print('k = ',k)
    print('Hamming Loss is',Hamming_Loss[count])
    print('One Error is',One_Error[count])
    print('Coverage is',Coverage[count])
    print('Ranking Loss is',Ranking_Loss[count])
    print('Average Precision is',Average_Precision[count])
    print('The running time is', end_time - start_time)
    print('\n')

k =  2
Hamming Loss is 0.24458333333333393
One Error is 0.6585
Coverage is 1.709
Ranking Loss is 0.2878958333333316
Average Precision is 0.5719277777777797
The running time is 9.753638505935669


k =  3
Hamming Loss is 0.2419166666666675
One Error is 0.6505
Coverage is 1.7065000000000001
Ranking Loss is 0.28797500000000015
Average Precision is 0.576172222222224
The running time is 9.296773195266724


k =  4
Hamming Loss is 0.23008333333333503
One Error is 0.642
Coverage is 1.666
Ranking Loss is 0.27553749999999766
Average Precision is 0.5820875000000013
The running time is 9.714772462844849


k =  5
Hamming Loss is 0.23158333333333492
One Error is 0.649
Coverage is 1.6635
Ranking Loss is 0.2753888888888863
Average Precision is 0.5798083333333355
The running time is 12.687468528747559


k =  6
Hamming Loss is 0.23550000000000137
One Error is 0.694
Coverage is 1.7065000000000001
Ranking Loss is 0.2866944444444422
Average Precision is 0.5604986111111134
The running time is 12.150251150131

In [ ]:
import time
distances = euclidean_distances(fs_svd_word)

X_train_sparse = sparse.csr_matrix(X_train)
svd_train = TruncatedSVD(n_components=200)
fs_svd_train = svd_train.fit_transform(X_train_sparse)
X_train_euclidean_distances = euclidean_distances(X_train_sparse)

true_labels = Y[8000:][:]
kk = 20
Hamming_Loss = np.zeros(kk)
One_Error = np.zeros(kk)
Coverage = np.zeros(kk)
Ranking_Loss = np.zeros(kk)
Average_Precision = np.zeros(kk)

for k in range(kk):
    start_time = time.time()
    P1, P0, Pe1, Pe0 = fit_ML_KNN(k, X_train, y_train, X_train_euclidean_distances)
    predict_labels, predict_rtl = predict_ML_KNN(k, P1, P0, Pe1, Pe0, X_test, Y_sum, distances)
    end_time = time.time()
    Hamming_Loss[k] = hloss(predict_labels, true_labels)
    One_Error[k] = one_error(predict_rtl, true_labels)
    Coverage[k] = coverage(predict_rtl, true_labels)
    Ranking_Loss[k] = rloss(predict_rtl, true_labels)
    Average_Precision[k] = avgprec(predict_rtl, true_labels)
    print('k = ',k)
    print('Hamming Loss is',Hamming_Loss[k])
    print('One Error is',One_Error[k])
    print('Coverage is',Coverage[k])
    print('Ranking Loss is',Ranking_Loss[k])
    print('Average Precision is',Average_Precision[k])
    print('The running time is', end_time - start_time)
    print('\n')